In [2]:
import pandas as pd #pip install pandas openpyxl

In [3]:
#input data dari explorasi data A
df = pd.read_excel("D:\Semester 7\TA\draft_multiapp\data\data_template.xlsx", 
                       engine='openpyxl',
                       sheet_name='Peg tetap',
                       usecols='A:AK',
                       nrows=2668,
                       )

In [4]:
# tingkat bunga aktuaria
tingkat_bunga = 0.0698
tingkat_bunga

0.0698

In [5]:
# tingkat kenaikan gaji
tingkat_kenaikan_gaji = 0.025
tingkat_kenaikan_gaji

0.025

In [6]:
# replacement ratio
replacement_ratio_max = 0.75
replacement_ratio_min = 0.4
pensiun_normal_ratio = 0.025
masa_kerja_minimal = 20

In [7]:
# --- BENEFIT DI DATA ---

#Kolom Gapok Terakhir
if "Gapok Terakhir" not in df:
    if tingkat_kenaikan_gaji > 0:
        df["Gapok Terakhir"] = df["Gaji Pokok"] * (1+tingkat_kenaikan_gaji) ** (df["Usia Pensiun (r) "]-df["Usia sekarang (x)"]-1)
        

In [8]:
#Kolom Benefit Pensiun Normal 1 Bulan
if "Benefit Pensiun Normal (1 Bulan)" not in df:
    if tingkat_kenaikan_gaji > 0:
        df["Benefit Pensiun Normal (1 Bulan)"] = 0.025 * df["Total Masa Kerja Harapan (MK)"] * df["Gapok Terakhir"]
#format nilai dalam rupiah
#st.session_state.df["Benefit Pensiun Normal (1 Bulan)"] = st.session_state.df["Benefit Pensiun Normal (1 Bulan)"].map('Rp{:,.2f}'.format)

In [9]:
#Kolom Benefit Pensiun Normal 1 Tahun
if "Benefit Pensiun Normal (1 Tahun)" not in df:
    if tingkat_kenaikan_gaji > 0:
        df["Benefit Pensiun Normal (1 Tahun)"] = df["Benefit Pensiun Normal (1 Bulan)"] * 12

In [12]:
# Kolom Benefit Pensiun Normal 1 Bulan (Taspen)
if "Benefit Pensiun Normal Taspen (1 Bulan)" not in df:
    if tingkat_kenaikan_gaji > 0:
        condition_1 = df["Total Masa Kerja Harapan (MK)"] >= 20
        condition_2 = df["Total Masa Kerja Harapan (MK)"] > replacement_ratio_max / pensiun_normal_ratio
        condition_3 = df["Total Masa Kerja Harapan (MK)"] < replacement_ratio_min / pensiun_normal_ratio

        # Use boolean indexing to set values based on conditions
        df.loc[condition_1 & condition_2, "Benefit Pensiun Normal Taspen(1 Bulan)"] = replacement_ratio_max * df.loc[condition_1 & condition_2, "Gapok Terakhir"]
        df.loc[condition_1 & condition_3, "Benefit Pensiun Normal Taspen(1 Bulan)"] = replacement_ratio_min * df.loc[condition_1 & condition_3, "Gapok Terakhir"]
        df.loc[(condition_1 & ~(condition_2 | condition_3)), "Benefit Pensiun Normal Taspen(1 Bulan)"] = pensiun_normal_ratio * df.loc[(condition_1 & ~(condition_2 | condition_3)), "Total Masa Kerja Harapan (MK)"] * df.loc[(condition_1 & ~(condition_2 | condition_3)), "Gapok Terakhir"]
    else:
        df["Benefit Pensiun Normal Taspen(1 Bulan)"] = 0
df.head(20)

# Specify the file path where you want to save the Excel file
excel_file_path = 'D:\Semester 7\TA\draft_multiapp\data\data_template2.xlsx'

# Export the DataFrame to an Excel file
df.to_excel(excel_file_path, index=False)